In [1]:
import random
import json
from aaalgo import inspect

# Tool Usage in OpenAI Chat Completion

Additional generation parameters.

- tools: a list of function specifications (see below.)
- tool_choice: "none", "auto", "required", or specification of one function.


In [2]:
MAX = 10

class User:
    def __init__ (self):
        self.reset()
        
    def reset (self):
        self.secret = random.randint(0, MAX)
        
    def guess (self, guess):
        if guess > self.secret:
            return "Too big!"
        elif guess < self.secret:
            return "Too small!"
        else:
            return "Bingo!"


In [3]:
from openai import OpenAI


TOOLS = [
    { "type": "function",
      "function": {
          "name": "reset",
          "description": "Start a new game.", # optional
          "parameters": {
              "type": "object",
              "properties": {
              },
              "required": [],
              "additionalProperties": False
          },          
          "strict": True
      },
    },
    { "type": "function",
      "function": {
          "name": "guess",
          "description": "Guess a number.",
          "parameters": {
              "type": "object",
              "properties": {
                  "n": { "type": "integer", "description": "The number you guessed."}
              },
              "required": ["n"],
              "additionalProperties": False
          },
          "strict": True
      }
    }    
]

SYSTEM_PROMPT = ("You are only allowed to respond by using the provided tool.")

PROMPT = ("Let's play the number guessing game.\n"
    f"I'll pick a secret number between 0 and {MAX},"
    "and you are to make guess attempts until you successfully find the correct number.\n"  
    "To each of your guess I'll respond by 'Too big!', 'Too small!' or 'Bingo!'\n"
    "Let's start!\n\n")


In [4]:
MODEL = 'gpt-4o-mini'

class Game:
    def __init__ (self):
        self.AI = OpenAI()
        self.user = User()
        self.offset = 0
        self.messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": PROMPT}
          ]        

    def showMessages (self):
        while self.offset < len(self.messages):
            msg = self.messages[self.offset]
            print("%s: %s" % (msg['role'], msg['content']))
            self.offset += 1

    def testStep (self):
        resp = self.AI.chat.completions.create(model=MODEL, tools=TOOLS, messages=self.messages, tool_choice='required')
        inspect(resp)

    def step (self):
        resp = self.AI.chat.completions.create(model=MODEL, tools=TOOLS, messages=self.messages, tool_choice='required')
        choice = resp.choices[0]
        for tool_call in choice.message.tool_calls:
            assert tool_call.type == 'function'
            function = tool_call.function
            name = function.name
            arguments = json.loads(function.arguments)

            if name == 'reset':
                self.user.reset()
                self.messages.append({'role': 'assistant', 'content': 'reset'})
                self.messages.append({'role': 'user', 'content': 'Please guess!'})
            elif name == 'guess':
                n = arguments['n']
                r = self.user.guess(n)
                self.messages.append({'role': 'assistant', 'content': str(n)})
                self.messages.append({'role': 'user', 'content': r})
            else:
                assert False, "AI is not following protocol."
            self.showMessages()
            
    def finish (self):
        self.messages.append({'role': 'user', 'content': 'We are going to stop here.  How many rounds have we played?'})
        resp = self.AI.chat.completions.create(model=MODEL, messages=self.messages)
        msg = resp.choices[0].message
        self.messages.append({'role': msg.role, 'content': msg.content})
        self.messages.append({'role': 'user', 'content': 'How many wrong guesses did you make in total?'})
        resp = self.AI.chat.completions.create(model=MODEL, messages=self.messages)
        msg = resp.choices[0].message
        self.messages.append({'role': msg.role, 'content': msg.content})
        
        self.showMessages()
        

In [5]:
game = Game()
game.testStep()

In [6]:
game = Game()

for i in range(10):
    game.step()
    
game.finish()

system: You are only allowed to respond by using the provided tool.
user: Let's play the number guessing game.
I'll pick a secret number between 0 and 10,and you are to make guess attempts until you successfully find the correct number.
To each of your guess I'll respond by 'Too big!', 'Too small!' or 'Bingo!'
Let's start!


assistant: reset
user: Please guess!
assistant: 5
user: Too small!
assistant: 7
user: Too small!
assistant: 8
user: Bingo!
assistant: reset
user: Please guess!
assistant: 5
user: Too big!
assistant: 4
user: Bingo!
assistant: reset
user: Please guess!
assistant: 5
user: Too small!
assistant: 7
user: Too small!
user: We are going to stop here.  How many rounds have we played?
assistant: We have played a total of 3 rounds.
user: How many wrong guesses did you make in total?
assistant: I made a total of 5 wrong guesses.
